### Base RD


In [3]:
import requests
import json
import re


def request_API(segment_id, page):
    url = "https://crm.rdstation.com/api/v1/organizations"
    params = {
        "token": "5b67dabde7b4bd01146f3d53",
        "page": page,
        "limit": 500,
        "order": "name",
        "direction": "asc",
        "organization_segment": segment_id,
    }

    response = requests.get(url, params=params)
    data = response.json()
    return data


all_companies = []

segment_ids = [
    "6407ca9e4a0478000c764571" "6407ca879a64fa001885e881",
    "6407ca8c53cbb5000c4dcd1b",
    "6407ca92ceb3cb00183993d6",
    "6407ca9988b49100262d7044",
    "640a651c7baa84000c626991",
    "6407caa96ff8ba0028be129f",
    "6407caa4ced60a00123844cc",
]

for segment_id in segment_ids:
    for page in range(1, 51):  # Assuming you want to iterate over 50 pages
        data = request_API(segment_id, page)
        companies = data["organizations"]
        for company in companies:
            filtered_fields = {}
            remove_company = False
            for field in company["custom_fields"]:
                if field["custom_field"]["label"] == "CNPJ":
                    value = field["value"]
                    if value is None:
                        remove_company = True
                        break
                    # Remove special characters and keep only numbers in CNPJ value
                    value = re.sub(r"\D", "", value)
                    # Format CNPJ value to have 14 digits
                    value = value.zfill(14)
                elif field["custom_field"]["label"] in ["FIXO 1", "FIXO 2", "FIXO 3"]:
                    value = field["value"]
                    if value is None:
                        remove_company = True
                        break
                else:
                    continue
                filtered_fields[field["custom_field"]["label"]] = value

            if not remove_company and any(filtered_fields.values()):
                filtered_company = {
                    "id": company["id"],
                    "name": company["name"],
                    **filtered_fields,
                }
                all_companies.append(filtered_company)

# Save filtered companies to a single file
with open("RD_leads.json", "w") as file:
    json.dump(all_companies, file)

### Base Exact


In [3]:
import requests
import json


def request_API(skip_num):
    url = "https://api.exactspotter.com/v3/Leads"
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }
    params = {"$skip": skip_num, "$orderby": "lead"}

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    return data


all_leads = []
skip = 0
while True:
    data = request_API(skip)
    leads = data["value"]
    if len(leads) == 0:
        break
    all_leads.extend(leads)
    skip += 500

# Filter and process the important data
filtered_leads = []
for lead in all_leads:
    filtered_lead = {
        "id": lead.get("id"),
        "lead": lead.get("lead"),
        "cnpj": lead.get("cnpj"),
    }
    if filtered_lead["cnpj"]:
        filtered_lead["cnpj"] = (
            filtered_lead["cnpj"]
            .replace(".", "")
            .replace("-", "")
            .replace("/", "")
            .zfill(14)
        )
    filtered_leads.append(filtered_lead)

# Save filtered leads to a JSON file
with open("exact_leads.json", "w") as file:
    json.dump(filtered_leads, file)


### De/Para


In [9]:
import json

# Load the RD leads from RD_leads.json
with open("RD_leads.json", "r") as rd_file:
    rd_leads = json.load(rd_file)

# Load the Exact leads from exact_leads.json
with open("exact_leads.json", "r") as exact_file:
    exact_leads = json.load(exact_file)

# Create a dictionary of RD leads based on CNPJ for efficient lookup
rd_leads_dict = {lead["CNPJ"]: lead for lead in rd_leads}

# Compare the leads and generate the desired output
output = []
for exact_lead in exact_leads:
    cnpj = exact_lead["cnpj"]
    if cnpj in rd_leads_dict:
        rd_lead = rd_leads_dict[cnpj]
        rd_fixed_1 = rd_lead.get("FIXO 1")
        rd_fixed_2 = rd_lead.get("FIXO 2")
        rd_fixed_3 = rd_lead.get("FIXO 3")
        if rd_fixed_1 or rd_fixed_2 or rd_fixed_3:
            output.append(
                {
                    "RD_lead_id": rd_lead["id"],
                    "RD_lead_name": rd_lead["name"],
                    "Exact_lead_id": exact_lead["id"],
                    "Exact_lead_name": exact_lead["lead"],
                    "CNPJ": cnpj,
                    "RD_Fixo_1": rd_fixed_1,
                    "RD_Fixo_2": rd_fixed_2,
                    "RD_Fixo_3": rd_fixed_3,
                }
            )

# Save the output to a JSON file
with open("leads_comparison.json", "w") as output_file:
    json.dump(output, output_file)

### Adicionando contatos Exact


In [1]:
import requests
import json

api_log = []


def add_contact_exact(Exact_lead_id, RD_Fixo_1, RD_Fixo_2):
    values = {
        "name": "Telefones Fixo",
        "leadId": Exact_lead_id,
        "jobTitle": "Fixo 1 e 2",
        "ddiPhone1": "55",
        "phone1": RD_Fixo_1,
        "ddiPhone2": "55",
        "phone2": RD_Fixo_2,
    }

    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }

    response = requests.post(
        "https://api.exactspotter.com/v3/personsAdd", json=values, headers=headers
    )

    # Log the information
    log_data = {
        "value": values,
        "status_code": response.status_code,
        "message": response.json(),
    }

    api_log.append(log_data)

    return response


# Load the leads comparison data from 'leads_comparison.json' file
with open("leads_comparison_4.json", "r") as file:
    leads_comparison = json.load(file)

# Iterate over the leads comparison list and add contacts using the Exact API
for lead in leads_comparison:
    Exact_lead_id = lead["Exact_lead_id"]
    RD_Fixo_1 = lead["RD_Fixo_1"]
    RD_Fixo_2 = lead["RD_Fixo_2"]

    print("Lead:")
    print(f"Exact_lead_id: {Exact_lead_id}")
    print(f"RD_Fixo_1: {RD_Fixo_1}")
    print(f"RD_Fixo_2: {RD_Fixo_2}")

    # confirmation = input("Do you want to add this contact? (y/n): ")
    # if confirmation.lower() == "y":
    response = add_contact_exact(Exact_lead_id, RD_Fixo_1, RD_Fixo_2)
    if response.status_code == 201:
        print("Contact added successfully.")
    else:
        print("Failed to add contact.")

    # Save log to a JSON file
    with open("comparison_logs_4.json", "w") as file:
        json.dump(api_log, file)
    # else:
    #     print("Contact skipped.")

Lead:
Exact_lead_id: 12410008
RD_Fixo_1: 11 36286904
RD_Fixo_2: 11 31656188
Contact added successfully.
Lead:
Exact_lead_id: 12410009
RD_Fixo_1: 11 36286904
RD_Fixo_2: 11 31656188
Contact added successfully.
Lead:
Exact_lead_id: 2033953
RD_Fixo_1: 1333521448
RD_Fixo_2: None
Contact added successfully.
Lead:
Exact_lead_id: 12433044
RD_Fixo_1: 1150174450
RD_Fixo_2: 1150175582
Contact added successfully.
Lead:
Exact_lead_id: 2033998
RD_Fixo_1: 1238825732
RD_Fixo_2: None
Contact added successfully.
Lead:
Exact_lead_id: 12459230
RD_Fixo_1: 1147421440
RD_Fixo_2: 1146421051
Contact added successfully.
Lead:
Exact_lead_id: 4888439
RD_Fixo_1: 1156217273
RD_Fixo_2: 1156213338
Contact added successfully.
Lead:
Exact_lead_id: 4840381
RD_Fixo_1: 11 37336654
RD_Fixo_2: 11 50110346
Contact added successfully.
Lead:
Exact_lead_id: 12433045
RD_Fixo_1: 1130443369
RD_Fixo_2: 1138420541
Contact added successfully.
Lead:
Exact_lead_id: 2025980
RD_Fixo_1: 1122566668
RD_Fixo_2: None
Contact added successfull

### Excluindo contatos Exact


In [1]:
import requests
import json

api_log = []


def get_contacts_exact(lead_id):
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }

    url = f"https://api.exactspotter.com/v3/Persons/{lead_id}"
    response = requests.get(url, headers=headers)

    return response


def delete_contact_exact(contact_id):
    headers = {
        "Content-Type": "application/json",
        "token_exact": "35fc1bd4-7d6e-4f40-bffd-123d2f46054a",
    }

    url = f"https://api.exactspotter.com/v3/PersonsRemove/{contact_id}"
    response_delete = requests.delete(url, headers=headers)

    return response_delete


# Load the exact_all_leads data from 'exact_all_leads.json' file
with open("./base/completa/exact_all_leads copy 3.json", "r") as file:
    exact_all_leads = json.load(file)

# Iterate over each lead in exact_all_leads
for lead in exact_all_leads:
    lead_id = lead["id"]

    print(f"Lead ID: {lead_id}")

    # confirmation = input("Delete contacts? (y/n): ")
    # if confirmation.lower() == "y":
    # Retrieve contacts for the lead
    response = get_contacts_exact(lead_id)
    if response.status_code == 200:
        contacts = response.json()["value"]

        # Iterate over each contact for the lead
        for contact in contacts:
            job_title = contact["jobTitle"]

            if job_title == "Telefones Fixo 1 e 2":
                contact_id = contact["id"]
                print(f"Deleting contact with ID: {contact_id}")

                # Delete the contact
                response_delete = delete_contact_exact(contact_id)

                # Log the information
                log_data = {
                    "get": {
                        "status_code_get": response.status_code,
                        "message_get": response.json()["value"],
                    },
                    "delete": {
                        "status_code_delete": response_delete.status_code,
                        "message_delete": "",
                    },
                }

                if response_delete.status_code == 204:
                    print("Contact deleted successfully.")
                    log_data["delete"][
                        "message_delete"
                    ] = "Contact deleted successfully."
                else:
                    print("Failed to delete contact.")
                    log_data["delete"]["message_delete"] = "Failed to delete contact."

                api_log.append(log_data)

                # Save log to a JSON file
                with open("delete_logs_4.json", "w") as file:
                    json.dump(api_log, file)

            else:
                print("Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').")
    else:
        print("Failed to retrieve contacts.")
    # else:
    #     print("Contacts retrieval and deletion skipped.")

Lead ID: 12411298
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 2031733
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 2033507
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 2032949
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 2016220
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 2032238
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 12404603
Contact skipped (jobTitle is not 'Telefones Fixo 1 and 2').
Lead ID: 12404604
Contact skipped (jobT